In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [182]:
def make_timedelta(df):
    df['seconds'] = df['time']
    df['time'] = pd.to_timedelta(df['time'], unit='s')

def get_highest_accuracy(df):
    highest_accuracy = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        er_df = df[df["error-rate"] == er]
        for j in range(len(er_df[(er_df["matches"] == np.max(er_df["matches"]))])):
            highest_accuracy.loc[i + j] = list(er_df[(er_df["matches"] == np.max(er_df["matches"]))].iloc[j])
            i = i + 1
        highest_accuracy.reset_index(drop = True, inplace = True)
    
    return highest_accuracy

def get_lowest_memory(df):
    lowest_memory = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["mem"] == np.min(df[df["error-rate"] == er]["mem"])])):
            lowest_memory.loc[i + j] = list(df[df["mem"] == np.min(df[df["error-rate"] == er]["mem"])].iloc[j])
            i = i + 1
    
    return lowest_memory
    
def get_fewest_matches(df):
    fewest_matches = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["matches"] == np.min(df[df["error-rate"] == er]["matches"])])):
            fewest_matches.loc[i + j] = list(df[df["matches"] == np.min(df[df["error-rate"] == er]["matches"])].iloc[j])
        i = i + 1
    
    return fewest_matches


def get_fastest(df):
    fastest = pd.DataFrame(columns=df.columns)
   
    i = 0
    for er in np.unique(df["error-rate"]):
        for j in range(len(df[df["seconds"] == np.max(df[df["error-rate"] == er]["seconds"])])):
            fastest.loc[i + j] = list(df[df["seconds"] == np.max(df[df["error-rate"] == er]["seconds"])].iloc[j])
            i = i + 1
    
    return fastest

# Mouse vs fly comparison

## DREAM-Stellar

### Clean log files

In [87]:
def read_time_df_default_params(filename):
    time = pd.read_csv(filename, sep = "\t")
    time["kmer-size"] = 0
    time["threshold"] = 0
    #time.drop(["fpr", "minimiser", "repeat-flag", "repeat-period", "repeat-length", "error-code"], axis = 1, inplace = True)
    cols = ["time", "mem", "bins", "max-er", "error-rate", "min-len","kmer-size", "threshold", "threads", "cmin", "cmax", "bin-entropy-cutoff", "cart-max-cap", "max-carts", "repeats", "matches"]
    time = time[cols]
    return time

def read_time_df(df):
    time = pd.read_csv(filename, sep = "\t")
    cols = ["time", "mem", "bins", "max-er", "error-rate", "min-len","kmer-size", "threshold", "threads", "cmin", "cmax", "bin-entropy-cutoff", "cart-max-cap", "max-carts", "repeats", "matches"]
    time = time[cols]
    return time

# set errors min_len kmer_sizes and thresholds
def add_params(df):
    for i in list(range(len(errors))):
        df.loc[(df["max-er"] == (errors[i]/min_len)) & (df["min-len"] == min_len), "kmer-size"] = kmer_sizes[i]
        df.loc[(df["max-er"] == (errors[i]/min_len)) & (df["min-len"] == min_len), "threshold"] = thresholds[i]

def get_stellar_only(df):
    stellar_only = df[(df["kmer-size"] == -1) | (df["threshold"] == -1)] 
    return stellar_only

def give_ungapped_kmer_stats(df):
    df["kmer-weight"] = df["kmer-size"]
    df["shape"] = df["kmer-size"].apply(lambda x: x * "1")
    return df

In [88]:
filename = "mouse/dream_masked/search_valik.time"
dream_masked = read_time_df_default_params(filename)

# from dream_masked/meta/
min_len = 100
kmer_sizes = [23, 23, 23, 18, 15, 14]
thresholds = [55, 32, 9, 11, 11, 20]
errors = list(range(1, 7))

add_params(dream_masked)

min_len = 50
kmer_sizes = [22, 14, -1]
thresholds = [7, 9, -1]
errors = list(range(1, 4))

add_params(dream_masked)

stellar_only = get_stellar_only(dream_masked)
dream_masked = dream_masked[(dream_masked["kmer-size"] != -1) & (dream_masked["threshold"] != -1)]
dream_masked = give_ungapped_kmer_stats(dream_masked)
dream_masked.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape
0,13.61,12554616,1024,0.01,1,100,23,55,32,0,254,0.25,2000,1024,0,15,23,11111111111111111111111
1,14.15,12555656,1024,0.01,1,100,23,55,32,0,254,0.75,2000,1024,0,15,23,11111111111111111111111
2,13.19,12554680,1024,0.01,1,100,23,55,32,0,254,0.50,2000,1024,0,15,23,11111111111111111111111
3,335.67,71710636,1024,0.06,6,100,14,20,32,0,254,0.25,2000,1024,6877,318,14,11111111111111
4,579.71,74472728,1024,0.06,6,100,14,20,32,0,254,0.75,2000,1024,6877,660,14,11111111111111


In [94]:
filename = "mouse/dream_masked_k/search_valik.time"
dream_masked_k = read_time_df(filename)
stellar_only = pd.concat([stellar_only, get_stellar_only(dream_masked_k)])
dream_masked_k = dream_masked_k[(dream_masked_k["kmer-size"] != -1) & (dream_masked_k["threshold"] != -1)]
dream_masked_k = give_ungapped_kmer_stats(dream_masked_k)
dream_masked_k.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape
0,234.56,63994764,1024,0.04,2,50,15,6,32,0,254,0.25,2000,1024,15483,842,15,111111111111111
4,234.01,65440112,1024,0.04,2,50,16,9,32,0,254,0.25,2000,1024,14463,1035,16,1111111111111111
5,232.90,65051396,1024,0.04,2,50,17,5,32,0,254,0.25,2000,1024,14570,1022,17,11111111111111111


In [95]:
filename = "mouse/dream_masked_k_t/search_valik.time"
dream_masked_k_t = read_time_df(filename)
stellar_only = pd.concat([stellar_only, get_stellar_only(dream_masked_k_t)])
dream_masked_k_t = dream_masked_k_t[(dream_masked_k_t["kmer-size"] != -1) & (dream_masked_k_t["threshold"] != -1)]
dream_masked_k_t = give_ungapped_kmer_stats(dream_masked_k_t)
dream_masked_k_t.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape
0,194.15,54655048,1024,0.04,2,50,16,11,32,0,50,0.25,2000,1024,808,2652,16,1111111111111111
1,120.17,39001736,1024,0.04,2,50,16,13,32,0,50,0.25,2000,1024,291,2620,16,1111111111111111
2,397.85,65668604,1024,0.04,2,50,16,7,32,0,50,0.25,2000,1024,12990,1362,16,1111111111111111
3,397.07,67473804,1024,0.04,2,50,16,9,32,0,50,0.25,2000,1024,3891,2441,16,1111111111111111
4,400.41,67468460,1024,0.04,2,50,16,9,32,0,150,0.25,2000,1024,4125,2416,16,1111111111111111


In [96]:
filename = "mouse/dream_masked_k_t2/search_valik.time"
dream_masked_k_t2 = read_time_df(filename)
stellar_only = pd.concat([stellar_only, get_stellar_only(dream_masked_k_t2)])
dream_masked_k_t2 = dream_masked_k_t2[(dream_masked_k_t2["kmer-size"] != -1) & (dream_masked_k_t2["threshold"] != -1)]
dream_masked_k_t2 = give_ungapped_kmer_stats(dream_masked_k_t2)
dream_masked_k_t2.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape
3,18.08,12552528,1024,0.02,1,50,17,17,32,0,254,0.25,2000,1024,6,1630,17,11111111111111111
4,247.49,64333616,1024,0.04,2,50,15,6,32,0,254,0.25,2000,1024,15483,840,15,111111111111111
5,19.54,12547896,1024,0.02,1,50,18,15,32,0,254,0.25,2000,1024,9,1637,18,111111111111111111
6,23.66,14791576,1024,0.02,1,50,15,21,32,0,254,0.25,2000,1024,16,1625,15,111111111111111
8,274.28,71259196,1024,0.04,2,50,16,9,32,0,254,0.25,2000,1024,4189,2422,16,1111111111111111


In [119]:
filename = "mouse/dream_masked_s/search_valik.time"
dream_masked_s = read_time_df(filename)
dream_masked_s["shape"] = dream_masked_s["kmer-size"]

dream_masked_s["kmer-size"] = dream_masked_s["shape"].apply(lambda x: len(str(x)))
dream_masked_s["kmer-weight"] = dream_masked_s["shape"].apply(lambda x: str(x).count("1"))
dream_masked_s.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,shape,kmer-weight
0,323.91,66229452,1024,0.04,2,50,18,11,32,0,100,0.25,2000,1024,1330,2755,111101110111011111,15
1,116.81,29682348,1024,0.04,2,50,18,15,32,0,50,0.25,2000,1024,88,3199,111101110111011111,15
2,404.56,70145128,1024,0.04,2,50,18,9,32,0,50,0.25,2000,1024,9885,1770,111101110111011111,15
3,299.86,65248700,1024,0.04,2,50,18,11,32,0,50,0.25,2000,1024,1172,2765,111101110111011111,15
4,355.05,65467336,1024,0.04,2,50,18,7,32,0,50,0.25,2000,1024,14757,1000,111101110111011111,15


### Analyse results

In [151]:
stellar_only

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches
50,7139.02,111881708,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
51,7186.11,111881548,1024,0.06,3,50,-1,-1,32,0,254,0.99,2000,1024,0,5352
52,7201.86,111879956,1024,0.06,3,50,-1,-1,32,0,254,0.50,2000,1024,0,5352
53,7224.07,111788964,1024,0.06,3,50,-1,-1,32,0,254,0.75,2000,1024,0,5352
1,6958.86,111785748,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
2,6728.52,111846568,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
3,6939.26,111771296,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
0,6743.10,111804344,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
1,6704.65,111902032,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352
2,6943.56,111773224,1024,0.06,3,50,-1,-1,32,0,254,0.25,2000,1024,0,5352


In [157]:
dream = pd.concat([dream_masked, dream_masked_k, dream_masked_k_t, dream_masked_k_t2, dream_masked_s])
make_timedelta(dream)
dream = dream.sort_values('error-rate')
dream.head()

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape,seconds
0,0 days 00:00:13.610000,12554616,1024,0.01,1,100,23,55,32,0,254,0.25,2000,1024,0,15,23,11111111111111111111111,13.61
24,0 days 00:00:17.230000,12510828,1024,0.01,1,100,23,55,32,0,254,0.25,2000,1024,0,15,23,11111111111111111111111,17.23
25,0 days 00:00:17.130000,12514000,1024,0.01,1,100,23,55,32,0,254,0.99,2000,1024,0,15,23,11111111111111111111111,17.13
42,0 days 00:05:23.350000,71761100,1024,0.02,1,50,22,7,32,0,254,0.50,2000,1024,67,1639,22,1111111111111111111111,323.35
43,0 days 00:03:59.110000,70152000,1024,0.02,1,50,22,7,32,0,254,0.75,2000,1024,67,1637,22,1111111111111111111111,239.11


In [185]:
dream_l100 = dream[dream["min-len"] == 100]
dream_l100.reset_index(drop = True, inplace = True)
dream_l50 = dream[dream["min-len"] == 50]
dream_l50.reset_index(drop = True, inplace = True)
dream_l30 = dream[dream["min-len"] == 30]
dream_l30.reset_index(drop = True, inplace = True)

In [186]:
get_highest_accuracy(dream_l100)

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape,seconds
0,0 days 00:00:13.610000,12554616,1024,0.01,1,100,23,55,32,0,254,0.25,2000,1024,0,15,23,11111111111111111111111,13.61
1,0 days 00:00:17.230000,12510828,1024,0.01,1,100,23,55,32,0,254,0.25,2000,1024,0,15,23,11111111111111111111111,17.23
2,0 days 00:00:17.130000,12514000,1024,0.01,1,100,23,55,32,0,254,0.99,2000,1024,0,15,23,11111111111111111111111,17.13
3,0 days 00:00:19.310000,12541140,1024,0.01,1,100,23,55,32,0,254,0.75,2000,1024,0,15,23,11111111111111111111111,19.31
4,0 days 00:00:15.680000,12535232,1024,0.01,1,100,23,55,32,0,254,0.50,2000,1024,0,15,23,11111111111111111111111,15.68
5,0 days 00:00:14.150000,12555656,1024,0.01,1,100,23,55,32,0,254,0.75,2000,1024,0,15,23,11111111111111111111111,14.15
6,0 days 00:00:13.190000,12554680,1024,0.01,1,100,23,55,32,0,254,0.50,2000,1024,0,15,23,11111111111111111111111,13.19
7,0 days 00:00:22.110000,12516560,1024,0.02,2,100,23,32,32,0,254,0.75,2000,1024,0,19,23,11111111111111111111111,22.11
8,0 days 00:00:13.410000,12555836,1024,0.02,2,100,23,32,32,0,254,0.50,2000,1024,0,19,23,11111111111111111111111,13.41
9,0 days 00:00:21.280000,12514980,1024,0.02,2,100,23,32,32,0,254,0.99,2000,1024,0,19,23,11111111111111111111111,21.28


In [188]:
get_fastest(dream_l100)

,time,mem,bins,max-er,error-rate,min-len,kmer-size,threshold,threads,cmin,cmax,bin-entropy-cutoff,cart-max-cap,max-carts,repeats,matches,kmer-weight,shape,seconds
0,0 days 00:00:19.310000,12541140,1024,0.01,1,100,23,55,32,0,254,0.75,2000,1024,0,15,23,11111111111111111111111,19.31
1,0 days 00:00:23.870000,12493544,1024,0.02,2,100,23,32,32,0,254,0.50,2000,1024,0,19,23,11111111111111111111111,23.87
2,0 days 00:00:45.200000,26904248,1024,0.03,3,100,23,9,32,0,254,0.25,2000,1024,2,214,23,11111111111111111111111,45.20
3,0 days 00:01:13.960000,25548372,1024,0.04,4,100,18,11,32,0,254,0.50,2000,1024,98,368,18,111111111111111111,73.96
4,0 days 00:10:07.630000,74870832,1024,0.05,5,100,15,11,32,0,254,0.99,2000,1024,12657,552,15,111111111111111,607.63
5,0 days 00:09:39.710000,74472728,1024,0.06,6,100,14,20,32,0,254,0.75,2000,1024,6877,660,14,11111111111111,579.71
